# GUI

In [2]:
import matplotlib.pyplot as plt
import tkinter as tk
from PIL import ImageTk, Image, ImageDraw
import cv2 as cv
import pickle as pk

In [3]:
with open('dig_KNN_clf.pickle','rb') as f:
    dig_KNN_clf = pk.load(f)
with open('dig_MLP_clf.pickle','rb') as f:
    dig_MLP_clf = pk.load(f)

In [ ]:
def draw(event):
    #draw white line on black background
    offs = 10
    '''draws on the canvas and on an invisible image that contains the same drawing'''
    global draw, img
    canvas.create_oval((event.x-offs),(event.y-offs), event.x+offs, event.y+offs, fill='white', outline ='white')
    draw.ellipse([(event.x-offs),(event.y-offs), event.x+offs, event.y+offs], fill='white')
    

def del_img():
    global img, draw
    canvas.delete('all')
    #delete img by reinitializing it
    img = Image.new('L',(im_w, im_h))
    draw = ImageDraw.Draw(img)
    #delete prediction output
    entryPred.delete(0,tk.END)

def predict():
    '''predicts digit, befor that the image is resized and
    the elements are converted to contain values of 0-16
    '''
    global draw, img, dig_MLP_clf, dig_KNN_clf
    img.save('img.png')
    im = cv.imread('img.png',cv.IMREAD_GRAYSCALE)
    im = cv.GaussianBlur(im,(5,5),1)
    im = cv.resize(im, dsize = (28,28),interpolation=cv.INTER_AREA)
    im = im.reshape(1,-1)
    pred  = dig_KNN_clf.predict(im)
    #print prediction
    if pred<= 9:
        offset = 48 # 0--> 48
    elif pred <36:
        offset = 55 # A --> 65
    else:
        offset = 60# a --> 97
    entryPred.delete(0,tk.END)
    entryPred.insert(tk.INSERT,f'class: {pred[0]}, ascii: '+chr(offset + pred[0]))

#detecting the mean of user digits in percent
mean = 0
tries = 0
def yes():
    global mean,tries
    tries +=1
    mean = mean*(tries-1)/tries + 1/tries
    entryAcc.delete(0,tk.END)
    entryAcc.insert(tk.INSERT,str(round(mean*100,1))+'%')
    del_img()
    
def no():
    global mean,tries
    tries +=1
    mean = mean*(tries-1)/tries
    entryAcc.delete(0,tk.END)
    entryAcc.insert(tk.INSERT,str(round(mean*100,1))+'%')
    del_img()
    
def key_yes_no(event):
    key = event.char
    if key == 'n' or key == 'N':
        no()
    if key == 'y' or key == 'Y':
        yes()
    if key == 'r' or key == 'R':
        reset()
        
def reset():
    global mean,tries
    mean = 0
    tries = 0
    entryAcc.delete(0,tk.END)
m = tk.Tk()
m.minsize(300,300)

#window + canvas to draw on
im_w, im_h = (224,224)
canvas = tk.Canvas(m, width=im_w, height=im_h, bg = 'black')
canvas.grid(row = 0, column = 1, rowspan = 5)
canvas.x_old = None
canvas.y_old = None
canvas.x_new = None
canvas.y_new = None
canvas.winfo_height
m.bind('<B1-Motion>', draw)
#image to store later (invisible)
img = Image.new('L',(im_w, im_h))
draw = ImageDraw.Draw(img)
#Buttons
buttonDelete = tk.Button(m, text='delete',width=10,height = 1,command=del_img)
buttonDelete.grid(row = 0, column = 0)

buttonPred = tk.Button(m, text='predict',width=10,height = 1,command=predict)
buttonPred.grid(row = 1, column = 0)
labelPred =tk.Label(m, text='Prediction:')
labelPred.grid(row = 2, column = 0)
entryPred = tk.Entry(m,width = 20, font = 15)
entryPred.grid(row = 3, column = 0)

buttonYes = tk.Button(m, text='Yes [y]',width=10,height = 1,bg ='green', command=yes)
buttonYes.grid(row = 4, column = 0)
buttonNo = tk.Button(m, text='No [n]',width=10,height = 1,bg ='red', command=no)
buttonNo.grid(row = 5, column = 0)
#bind 'n' and 'y' keyboard
m.bind('<Key>', key_yes_no)


labelAcc =tk.Label(m, text='Accuracy:')
labelAcc.grid(row = 6, column = 0)
entryAcc = tk.Entry(m,width = 10, font = 15)
entryAcc.grid(row = 7, column = 0)
buttonReset = tk.Button(m, text='Reset % [r]',width=10,height = 1,bg ='grey', command=reset)
buttonReset.grid(row = 7, column = 1)


m.mainloop()